In [5]:
import re # regular expression
import os # operating system
import shutil # shell utilities
import fnmatch # file name matching

#### Replace NcRd functions by newly-added get_met functions in flexgrid_read_mod.F90 (geosfp_read_mod.F90 equivalent)

========2D fields==========

CALL NcRd( Q, fCN, TRIM(v_name), st3d, ct3d )<br>
CALL NcRd( Q, fA1, TRIM(v_name), st3d, ct3d )<br>
=><br>
CALL Get_Met_2D( Q, TRIM(v_name) )

CALL NcRd( Q2, fI3_1, TRIM(v_name), st3d, ct3d ) <br>
CALL NcRd( Q2, fI3_2, TRIM(v_name), st3d, ct3d ) <br>
=><br>
CALL Get_Met_2D( Q2, TRIM(v_name) )

========3D fields==========

CALL NcRd( Q, fA3cld, TRIM(v_name), st4d, ct4d )<br>
CALL NcRd( Q, fA3dyn, TRIM(v_name), st4d, ct4d )<br>
CALL NcRd( Q, fA3mstC, TRIM(v_name), st4d, ct4d ) <br>
=><br> 
CALL Get_Met_3D( Q, TRIM(v_name) )

CALL NcRd( Q3, fI3_1, TRIM(v_name), st4d, ct4d) <br>
CALL NcRd( Q3, fI3_2, TRIM(v_name), st4d, ct4d) <br>
=><br> 
CALL Get_Met_3D( Q3, TRIM(v_name) )

========3D edge fields==========

CALL NcRd( Qe, fA3mstE, TRIM(v_name), st4d, ct4d ) <br>
=><br> 
CALL Get_Met_3De( Qe, TRIM(v_name) )

========transfer functions==========

CALL Transfer_3d( Q, State_Met%SOME_VAR_NAME )<br>
=><br>
State_Met%SOME_VAR_NAME = Q

CALL Transfer_3d_Lp1( Qe, State_Met%SOME_VAR_NAME )<br>
=><br> 
State_Met%SOME_VAR_NAME = Qe

========array size==========

Q(IIPAR,JJPAR,LGLOB)<br>
=><br>
Q(IIPAR,JJPAR,LLPAR)

Q(IIPAR,JJPAR,LGLOB+1)<br>
=><br>
Q(IIPAR,JJPAR,LLPAR+1)

(This can be simply done by :%s/JJPAR,LGLOB/JJPAR,LLPAR/g in vim)

In [6]:
def exist(pattern_list,line):
    # pattern_list: a list of string
    # line: the line to match
    result = False
    for pattern in pattern_list:
        if pattern in line:
            result = True   
    return result

def allmatch(pattern_list,line):
    # pattern_list: a list of string
    # line: the line to match
    result = True
    for pattern in pattern_list:
        if not pattern in line:
            result = False
    return result

In [7]:
# warning: this cell will modify the code files!
# use git reset --hard to recover the orginal state

codefile = '../Code_v11-02_FlexGrid/GeosCore/flexgrid_read_mod.F90'

# read the file line by line
with open(codefile,"r",encoding = "ISO-8859-1") as file:

    # read all lines, modify lines in-memory, and finally overwrite original file
    # this approach could be relative slow but clean.
    lines = file.readlines()

    for n,line in enumerate(lines):

        if "CALL NcRd" in line:
            
            if "!" in line:
                # DO NOT convert comment to real code!!
                continue
        
            if exist(['fCN','fA1'],line):
                # 2D data
                templine = '    CALL Get_Met_2D( Q, TRIM(v_name) )\n'
                
            if exist(['fA3cld','fA3dyn','fA3mstC'],line):
                # 3D data
                templine = '    CALL Get_Met_3D( Q, TRIM(v_name) )\n'
                
            if exist(['fA3mstE'],line):
                # 3D edge data
                templine = '    CALL Get_Met_3De( Qe, TRIM(v_name) )\n'  
                
            if  allmatch(['fI3','Q2'],line):
                # 2D I3 data
                templine = '    CALL Get_Met_2D( Q2, TRIM(v_name) )\n'
                
            if allmatch(['fI3','Q3'],line):
                # 3D I3 data
                templine = '    CALL Get_Met_3D( Q3, TRIM(v_name) )\n'
            
            print(lines[n-1],end='')
            print(line,end='')
            print(templine)
            lines[n]=templine

# open the same file to overwrite
with open(codefile,"w",encoding = "ISO-8859-1") as file:
    for line in lines:
        file.write(line)

    v_name = "FRLAKE"
    CALL NcRd( Q, fCN, TRIM(v_name), st3d, ct3d )
    CALL Get_Met_2D( Q, TRIM(v_name) )

    v_name = "FRLAND"
    CALL NcRd( Q, fCN, TRIM(v_name), st3d, ct3d )
    CALL Get_Met_2D( Q, TRIM(v_name) )

    v_name = "FRLANDIC"
    CALL NcRd( Q, fCN, TRIM(v_name), st3d, ct3d )
    CALL Get_Met_2D( Q, TRIM(v_name) )

    v_name = "FROCEAN"
    CALL NcRd( Q, fCN, TRIM(v_name), st3d, ct3d )
    CALL Get_Met_2D( Q, TRIM(v_name) )

    v_name = "PHIS"
    CALL NcRd( Q, fCN, TRIM(v_name), st3d, ct3d )
    CALL Get_Met_2D( Q, TRIM(v_name) )

    v_name = "ALBEDO"
    CALL NcRd( Q, fA1, TRIM(v_name), st3d, ct3d )
    CALL Get_Met_2D( Q, TRIM(v_name) )

    v_name = "CLDTOT"
    CALL NcRd( Q, fA1, TRIM(v_name), st3d, ct3d )
    CALL Get_Met_2D( Q, TRIM(v_name) )

    v_name = "EFLUX"
    CALL NcRd( Q, fA1, TRIM(v_name), st3d, ct3d )
    CALL Get_Met_2D( Q, TRIM(v_name) )

    v_name = "EVAP"
    CALL NcRd( Q, fA1, TRIM(v_name), st3d, ct3d )
    CALL Get_Met_2D( Q, TRIM(v_nam

In [8]:
# warning: this cell will modify the code files!
# use git reset --hard to recover the orginal state

codefile = '../Code_v11-02_FlexGrid/GeosCore/flexgrid_read_mod.F90'

# read the file line by line
with open(codefile,"r",encoding = "ISO-8859-1") as file:

    # read all lines, modify lines in-memory, and finally overwrite original file
    # this approach could be relative slow but clean.
    lines = file.readlines()

    for n,line in enumerate(lines):

        if "CALL Transfer_3d" in line:
            
            if "!" in line:
                # DO NOT convert comment to real code!!
                continue
            
            print(line,end='')
            
            target = line.split()[3]
            source = line.split()[2][:-1]
            
            templine = '    '+target+" = "+source+'\n'
            print(templine)
            
            lines[n]=templine

# open the same file to overwrite
with open(codefile,"w",encoding = "ISO-8859-1") as file:
    for line in lines:
        file.write(line)

    CALL Transfer_3d( Q, State_Met%QI )
    State_Met%QI = Q

    CALL Transfer_3d( Q, State_Met%QL )
    State_Met%QL = Q

    CALL Transfer_3d( Q, State_Met%TAUCLI )
    State_Met%TAUCLI = Q

    CALL Transfer_3d( Q, State_Met%TAUCLW )
    State_Met%TAUCLW = Q

    CALL Transfer_3d( Q, State_Met%DTRAIN )
    State_Met%DTRAIN = Q

    CALL Transfer_3d( Q, State_Met%OMEGA )
    State_Met%OMEGA = Q

    CALL Transfer_3d( Q, State_Met%RH )
    State_Met%RH = Q

    CALL Transfer_3d( Q, State_Met%U )
    State_Met%U = Q

    CALL Transfer_3d( Q, State_Met%V )
    State_Met%V = Q

    CALL Transfer_3d( Q, State_Met%DQRCU )
    State_Met%DQRCU = Q

    CALL Transfer_3d( Q, State_Met%DQRLSAN )
    State_Met%DQRLSAN = Q

    CALL Transfer_3d( Q, State_Met%REEVAPCN )
    State_Met%REEVAPCN = Q

    CALL Transfer_3d( Q, State_Met%REEVAPLS )
    State_Met%REEVAPLS = Q

    CALL Transfer_3d_Lp1( Qe, State_Met%CMFMC )
    State_Met%CMFMC = Qe

    CALL Transfer_3d_Lp1( Qe, State_Met%PFICU )
    St